# Analisador de sentimentos

Def.:
> "A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P, if its performance at tasks in T, as measured by P, improves with experience E."  

(Tom Mitchell) *[1][2]*

Por essa definição, temos um grande leque de problemas que podem ser resolvidos por algoritmos de machine learning. Um sistema analisador de sentimentos é um subproblema de classificação em machine learning bem definido, pois temos a tarefa "classificar um texto como positivo ou negativo" com uma medida de performance sendo, por exemplo, a acurácia - quantidade de predições corretas, isto é, 1-((#FN+#FP)/total) ou (#TP+#TN)/total), onde TN = true negative, TP true positive, FN = false negative e FP = false positive, essa notação e métricas de performance além da acurácia podem ser vistas em [5] - e experiênca sendo o histórico conhecido de análises (podendo ser realimentada com feedbacks futuros, a fim de melhorar o modelo). Dessa forma, temos um sistema de machine learning de classificador binário (pois temos duas classes possíveis: positivo e negativo).

Um classificador desse tipo pode ser implementado com vários algoritmos, por exemplo, com perceptron que separa linearmente o espaço, isto é, baseado em uma função linear que divide todo o espaço em 2 classes possíveis, é possível decidir em favor de uma classe ou de outra, dependendo de onde se encontra geometricamente algum ponto (um vetor de caracteristicas aplicado na mesma função), usando a variação do perceptron chamada de regressão logística é possível saber qual a probabilidade de uma entrada nova pertencer a uma classe ou outra de forma direta (não é um algoritmo probabilístico, ainda é um separador linear de espaços), nessa categoria é possível também aplicar algoritmos mais robustos como SVM que, caso o espaço seja linearmente separável, cria o melhor plano que divide as classes como um problema de otimização que maximiza a chamada "margem" entre o plano separador e os chamados "vetores de suporte" de cada classe (intuitivamente: cria a melhor divisão de espaços possível para uma generalização do problema, isto é, para amostras desconhecidas), o SVM também pode ter uma boa performance em espaços não separáveis com a ideia de kerneis que mapeiam as entradas para espaços de altas dimensões e aplicam a mesma ideia generalizada (é possível usar kerneis bem conhecidos ou criar/adaptar kerneis dependendo do problema).

Apesar das diversas formas não probabilísticas disponíveis para abordar esse tipo de tarefa, um dos algoritmos probabilisticos mais simples, o classificador Naïve Bayes, geralmente é mais utilizado *[3]*.

Classificadores probabilísticos criam uma distribuição de probabilidade sobre todas as classes possíveis, no caso específico do problema, para as classes "positivo" e "negativo", isto é, se x é um vetor de entrada a ser classificado, usando a notação latex para "\in"="pertence", P(x \in "positivo") \in [0, 1], P(x \in "negativo") \in [0, 1], P(x \in "positivo") + P(x \in "negativo") = 1, e o classificador escolhe a classe com maior pro probabilidade como a predição para x.

Para o caso de classificação de texto, o classificador Naïve Bayes calcula a probabilidade a posteriori de uma classe baseada na distribuição das palavras no documento inteiro. O vetor de entrada, nesse caso, é conhecido como bag of words, que basicamente é um vetor de contagem de palavras (ou algumas variações na mesma ideia, por exemplo normalizando a frequência das palavras).

Naïve Bayes recebe esse nome pois usa fortemente o teorema de Bayes com suposição (dita ingênua ou burra) de independência entre as componentes do vetor de entrada, isto é, afirma que P(Classe_k | vetor de entrada) = (P(vetor de entrada | Classe_k) * P(Classe_k)) / P(vetor de entrada), para qualquer classe k, dessa forma o denominador é uma constante para o cálculo (se repete para o cálculo de qualquer classe) - o que facilita computacionalmente as coisas -, assumindo que a ocorrência e quantidade de uma palavra específica é independente da ocorrência e quantidade de uma outra palavra específica - simplificando o cálculo desse algoritmo mais ainda, pois P(vetor de entrada | Classe_k) = P(componente 1 do vetor de entrada | Classe_k)*P(componente 2 do vetor de entrada | Classe_k)*...*P(componente  do vetor de entrada | Classe_k). Logo esse algoritmo é computacionalmente simples e tem uma boa performance em relação a outros algoritmos possíveis.

Um bom algoritmo para a tarefa de análise de sentimentos tem performance de pelo menos 70% de acurácia [6], visto que esta tarefa é complexa mesmo para seres humanos (essa mesma acurácia é esperada mesmo para seres humanos), pois há muitas variáveis envolvidas, tais como detecção de sarcásmo, termos neutros, entre outros... então, a acuárácia esperada será em torno desses valores.

Nesse projeto será utilizado, portanto, o algoritmo Naïve Bayes. A distribuição de probabilidade utilizada para os cálculos será a multinomial, pois tem uma performance superior para tarefas de classificação de textos [4]. Os seguintes passos serão seguidos para a conclusão do projeto:

Nesse texto, seria interessante ignorar as bibliotecas que serão utilizadas (sys para leitura da entrada padrão, numpy para facilitar em tarefas como randomização da entrada, sklearn (naive_bayes para o algoritmo em si, metrics para avaliação da performance e CountVectorizer que será o responsável por transformar a entrada em um vetor de características para algoritmo), bem como a validação da entrada padrão, que basicamente apenas verifica se há alguma frase para ser testada, esse código ficará disponível no código em python completo juntamente com esse texto no GitHub. Mas o Juptyter interpreta o código linha a linha, então esse passo será mostrado na abaixo.

In [63]:
import sys

import numpy as np

from sklearn import naive_bayes
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

if len(sys.argv) < 2 or len(sys.argv) > 2:
	# sys.exit(1) para o Jupter rodar, nesse caso será adicionada uma sentenca negativa
	testInput = 'its a bad film'
else:
	testInput = sys.argv[1]   
print("frase que sera testada ao final do algoritmo: %s" % testInput) # apenas para o Jupyter   

frase que sera testada ao final do algoritmo: its a bad film


* Carregar os dados

In [53]:
allReviews = [] #vetor de sentencas, contera todas as entradas (negativas ou positivas)

* O arquivo rt-polarity.neg será carregado como vetores de reviews negativos, definindo o label 0 para cada review.

In [54]:

with open("rt-polarity.neg", 'r') as file:
	for line in file:
		allReviews.append([line, 0])

* O arquivo rt-polarity.pos será carregado como vetores de reviews positivos, definindo o label 1 para cada review.


In [55]:
with open("rt-polarity.pos", 'r') as file:
	for line in file:
		allReviews.append([line, 1])

* Separar os dados para treinamento dos dados de teste.
    * Randomizar a entrada para diminuir erros amostrais;


In [56]:
np.random.shuffle(allReviews) 

* usar 90% para treinamento e 10% para teste. Nota: há diversas formas comuns para essa separação, por exemplo: (N-1) / 1, 80%/20%, Cross-validation Exaustiva (todas as variações possíveis), 50%/50%, entre outras... foi escolhido essa mais comum, por simplicidade, com uma amostra de 10 mil+.


In [57]:
trainReviews = allReviews[:int(len(allReviews)*0.9)]
testReviews =  allReviews[int(len(allReviews)*0.9):]

* Transformar as palavras em "bag of words", ou vetor de contagem de frequências de 'tokens'.
    * Nessa etapa é possível tratar a entrada, por exemplo, retirar palavras julgadas desnecessárias ou que atrapalhem a tarefa, manter radicais das palavras, fazer conversões, etc... Para a proposta desse projeto, será feita tokenização com retirada de "stop words" do idioma inglês, caracteres não unicode e será ignorados termos que aparecem apenas uma vez.

In [58]:
vectorizer = CountVectorizer(stop_words='english', strip_accents='unicode', min_df = 2)

train_features = vectorizer.fit_transform([r[0] for r in trainReviews]) 
test_features = vectorizer.transform([r[0] for r in testReviews])   

* Treinar o sistema com os dados separados para treinamento no item 2.2.
    * Como dito anteriormente, usaremos o modelo multinomial do classificador de Naïve Bayes, que criará um modelo de inferência da seguinte maneira: seja C_k a classe (negativa|positiva), x o vetor de características (vetor de frequência de palavras), o modelo de bayes calcula P( C_k | x ) = (P( x | C_k )P( C_k ))/P( x ) = pela hipótese ingênua = (P( x1 | C )P( x2 | C )...P( xn | C_k )P( C_k ))/P( x ) e, como para qualquer classe o denominador é irrelevante para a escolha da classe que maximiza o valor (é constante para toda classe), basta que o modelo calcule (P( x1 | C_k )P( x2 | C_k )...P( xn | C_k )P( C_k )) para cada classe (negativo|positivo). Usando as contagens de cada componente de x (cada token), e o número de sentenças de cada classe, o modelo está pronto. Por exemplo, P( x1 | C_k ) = número de ocorrências do token na primeira posição do vetor x dividido pelo número total de sentenças da classe C_k no conjunto de treinamento, e P( C_k ) = número de ocorrencias de sentenças da classe C dividido pelo total de sentenças no conjunto de treinamento. (se a ideia fosse implementar o algoritmo sem uso de biblioteca, haveria um problema técnico pensando nas multiplicaçoes das frequências diretamente como estimador, pois, caso uma palavra não exista no conjunto dessa classe, esse termo na produtória zeraria toda a produtória, para resolver isso, usa-se o estimador de Laplace, que basicamente adiciona 1 para cada termo possível e soma ao denominador o total de termos para que a fórmula nunca passe de 1... mas esse problema já é resolvido usando a implementação naive_bayes da biblioteca sklearn).

In [59]:
nb = naive_bayes.MultinomialNB()
nb.fit(train_features, [r[1] for r in trainReviews])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

* Nesse modelo, a saída é definida como a classe que maximiza a fórmula, ou seja, para o caso específico, onde as classes são 0 (negativo) e 1 (positivo), se P( C_0 | x ) > P( C_1 | x ), o modelo infere que a saída é 0, caso contrário, 1.

* Avaliar a acurácia do sistema com os dados separados para teste no item 2.2.
    * Basta realizar a predição, isto é, aplicar cada vetor de características (frequência dos tokens) do conjunto de teste no modelo criado no item 4 e ver os resultados das predições feitas para o conjunto de dados de teste, quantos foram preditos corretamente, isto é, quantos eram negativos (ou positivos) e foram avaliados como negativos (ou positivos) e dividir pelo total de predições feitas (total do conjunto de testes).


In [60]:
predictions = nb.predict(test_features)
testLabels = [r[1] for r in testReviews]

acuracy = metrics.accuracy_score(testLabels, predictions)
print("Acuracia do modelo: %0.5f" % acuracy)	

Acuracia do modelo: 0.78913


* Classificar uma entrada nova, como teste.
    * Para testar o modelo com entradas novas, será usada a entrada padrão, então bastará executar o programa passando uma sentença em inglês, para que o sistema diga se a mesma é uma sentença com sentimento positivo ou negativo. 


In [61]:
inputPrediction = nb.predict( vectorizer.transform( [testInput] ) )[0]
if( inputPrediction == 0 ):
	print('Your input (%s) is a negative sentence' % testInput)
else:
	print('Your input (%s) is a positive sentence' % testInput)

Your input (its a bad film) is a negative sentence


[1] *Quote Catalog (https://quotecatalog.com/quote/tom-mitchell-a-computer-prog-81zBYB1)*

[2] *Machine Learning, Tom M. Mitchell (http://dl.acm.org/citation.cfm?id=541177)*

[3] *Sentiment analysis algorithms and applications: A survey (http://www.sciencedirect.com/science/article/pii/S2090447914000550)*

[4] *A Comparison of Event Models for Naive Bayes Text Classification (http://www.cs.cmu.edu/~knigam/papers/multinomial-aaaiws98.pdf)*

[5] *Confusion Matrix Terminology http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/*

[6] *The Problem With Automated Sentiment Analysis http://www.freshminds.net/2010/05/the-problem-with-automated-sentiment-analysis/*